# Document Normalizer annotator notebook

# Set up Colab environment

In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-12-23 14:48:19--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-12-23 14:48:20--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-12-23 14:48:20--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

# Start Spark NLP session

In [3]:
# Import Spark NLP
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp

spark =sparknlp.start()

In [4]:
spark

# Regex Tokenizer annotator

In [5]:
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp

content = "1. T1-T2 DATE**[12/24/13] $1.99 () (10/12), ph+ 90%"
pattern = "\\s+|(?=[-.:;*+,$&%\\[\\]])|(?<=[-.:;*+,$&%\\[\\]])"

df = spark.createDataFrame([content], StringType()).withColumnRenamed("value", "text")

df.show()

documentAssembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentenceDetector = SentenceDetector() \
      .setInputCols(["document"]) \
      .setOutputCol("sentence")

regexTokenizer = RegexTokenizer() \
      .setInputCols(["sentence"]) \
      .setOutputCol("regexToken") \
      .setPattern(pattern) \
      .setPositionalMask(False)

docPatternRemoverPipeline = \
  Pipeline() \
    .setStages([
        documentAssembler,
        sentenceDetector,
        regexTokenizer])

ds = docPatternRemoverPipeline.fit(df).transform(df)

ds.show(10, False)

+--------------------+
|                text|
+--------------------+
|1. T1-T2 DATE**[1...|
+--------------------+

+---------------------------------------------------+---------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------